In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Generate dummy data
np.random.seed(42)
# Initialize data
num_samples = 1000
num_features = 5
importance_weight = 100.0 # Scale up the weights x times
importance_bias = 10.0 # Scale up the bias x times
importance_noise = 10.0 # Scale up the noise x times
outlier_value = 10000.0 # Scaling the outlier x times


data_input = np.random.rand(num_samples, num_features)  # n samples of m features each
true_weights = np.random.rand(num_features) * importance_weight # True feature weights
true_biases = np.random.rand(num_features) * importance_bias # True bias

def f(x):
  # Create a mere linear equation
  f = true_weights * x + true_biases # y = w_1 * x_1 + b_1 + ... + w_n * x_n + b_n
  f = np.sum(f, -1)
  # Add noise to the data
  f = f + np.random.uniform(low=-1.0, high=1.0, size=(num_samples)) * importance_noise
  # Create outlier with a chance of 5%
  f = f + np.random.uniform(low=-1.0, high=1.0, size=(num_samples)) * np.random.choice([True, False], size = num_samples, p=[0.05, 0.95]) * outlier_value
  return f


data_output = f(data_input)

In [ ]:
# Define the model
network = Sequential()
# Model will only have
network.add(Dense(1, input_shape=(num_features,)))
# Compile the model with mean squared error loss and SGD optimizer
network.compile(loss="mean_squared_error", optimizer=SGD(learning_rate=0.01))

# Train the model
network.fit(data_input, data_output, epochs=150, verbose=1)

Epoch 1/150
32/32 [==============================] - 1s 2ms/step - loss: 1730540.7500
Epoch 2/150
32/32 [==============================] - 0s 2ms/step - loss: 1721693.7500
Epoch 3/150
32/32 [==============================] - 0s 2ms/step - loss: 1720596.6250
Epoch 4/150
32/32 [==============================] - 0s 2ms/step - loss: 1721965.6250
Epoch 5/150
32/32 [==============================] - 0s 2ms/step - loss: 1721153.2500
Epoch 6/150
32/32 [==============================] - 0s 2ms/step - loss: 1720106.3750
Epoch 7/150
32/32 [==============================] - 0s 2ms/step - loss: 1720328.7500
Epoch 8/150
32/32 [==============================] - 0s 2ms/step - loss: 1720711.1250
Epoch 9/150
32/32 [==============================] - 0s 2ms/step - loss: 1720043.1250
Epoch 10/150
32/32 [==============================] - 0s 2ms/step - loss: 1719771.5000
Epoch 11/150
32/32 [==============================] - 0s 2ms/step - loss: 1719816.5000
Epoch 12/150
32/32 [==============================] 

In [ ]:
# Retrieve the feature weights and total biases
weights, biases = network.layers[-1].get_weights()
print(f"Weights: {weights.flatten()}")
print(f"Biases: {biases}")

# The initalize weights and biases
print(true_weights)
print(true_biases)

Weights: [ 57.69515   45.928505 -56.552544  13.436747 186.17789 ]
Biases: [42.657295]
[39.36355203 47.34356594 85.45473932 34.00043861 86.96496848]
[0.88134431 7.76798437 8.47547633 1.81817653 4.30346532]


In [ ]:
# Generate test data
test_input = np.random.rand(20, num_features)  # 20 samples
test_input = np.sort(test_input)

def f(x):
  f = true_weights * x + true_biases
  f = np.sum(f, -1)
  return f

test_output = f(test_input) # Create linear data without noise or outlier
predictions = network.predict(test_input)
percentage_errors = [abs(predictions[i][-1] - test_output[i]) / abs(test_output[i]) * 100
                     for i in range(len(test_output))] # Calculate the percentage of difference
for i in range(len(test_input)):
    print(f"Input: {test_input[i]}, True Output: {test_output[i]}, Predicted Output: {predictions[i][-1]},
    Loss: {percentage_errors[i]:.2f}%")

1/1 [==============================] - 0s 18ms/step
Input: [0.30521453 0.40823752 0.49961841 0.66336261 0.97374258], True Output: 204.51906865624767, Predicted Output: 240.96450805664062, Loss: 17.82%
Input: [0.17170999 0.24218345 0.28046322 0.52983389 0.73110298], True Output: 147.03323407435755, Predicted Output: 191.060791015625, Loss: 29.94%
Input: [0.11793867 0.28329653 0.28998739 0.62890514 0.6549411 ], True Output: 144.42198021138734, Predicted Output: 176.45962524414062, Loss: 22.18%
Input: [2.42934435e-04 1.26936058e-01 2.33609032e-01 8.67354639e-01
 9.33615378e-01], True Output: 159.91088424246752, Predicted Output: 220.7630615234375, Loss: 38.05%
Input: [0.37360566 0.60857159 0.68146061 0.7783719  0.97090471], True Output: 235.89856481282737, Predicted Output: 244.84475708007812, Loss: 3.79%
Input: [0.06353572 0.2878795  0.39624735 0.53898087 0.96381766], True Output: 175.38185321487703, Predicted Output: 223.81976318359375, Loss: 27.62%
Input: [0.12027096 0.37393871 0.64485

In [ ]:
## Only use for one feature
## Plot the test data and the linear equation that model predicts
# sorted_indices = np.argsort(test_input.flatten())
# sorted_test_input = test_input[sorted_indices]
# sorted_test_output = test_output[sorted_indices]
# plt.scatter(sorted_test_input, sorted_test_output)
# w = weights.flatten()
# b = biases
# plt.plot(sorted_test_input, w * sorted_test_input + b, color='red')
# plt.plot()
# plt.show()